In [ ]:
!pip install pyenchant
import nltk
nltk.download("words")
nltk.download("stopwords")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import enchant
import string
import regex
import re
from nltk.stem import PorterStemmer
from nltk.corpus import words as eng_words
from nltk.corpus import stopwords

In [414]:
class TextCleaning:
   def __init__(self,stemming: bool):
       self._ps = PorterStemmer()
       if type(stemming) is not bool:
        raise TypeError("Parameter < stemming > must be < bool >")
       else:
        self._stem = stemming
       self._eng = enchant.Dict("en_US")
       self._sw = self._imp_stop_words()
       self._double = np.squeeze(pd.read_csv(
           "https://raw.githubusercontent.com/NITHISHM2410/Text_Processing/NLP/Text%20Cleaning/double_identical.txt",
           header=None).values).tolist()
       self._notwords = np.squeeze(pd.read_csv("https://raw.githubusercontent.com/NITHISHM2410/Text_Processing/NLP/Text%20Cleaning/negative%20words.txt",
                                               header=None).values.tolist())
       self._notwords = self.ready_not_words()
       self._eng_words = self.ready_eng_words()
 
   def ready_eng_words(self):
       self._eng_words = eng_words.words()
       self._eng_words += self._notwords
       return self._eng_words

   def ready_not_words(self):
       self._notwords = [i.replace("'", "") for i in self._notwords]
       self._notwords.append("not")
       return self._notwords

   @staticmethod
   def _imp_stop_words():
       words = stopwords.words("english")
       words.append("hello")
       ntwords = [i for i in words if "n't" in i or i[-1] == 'n']
       ntwords.remove('between')
       ntwords.remove('again')
       ntwords.remove('on')
       ntwords.remove('an')
       ntwords.remove('won')
       ntwords.remove('when')
       ntwords.remove('than')
       ntwords.remove('then')
       ntwords = ntwords[5:]
       stop_words = set(words) - set(ntwords)
       stop_words.remove("not")
       return stop_words
   def not_list(self):
       return self._notwords

   def _remove_consec_duplicates(self,s):
       new_s = ""
       prev = ""
       for c in s:
           if len(new_s) == 0:
               new_s += c
               prev = c
           if c == prev:
               continue
           else:
               new_s += c
               prev = c
       return new_s

   def clean(self,series):
       for i in range(len(series)):
           series[i] = self._cleaner(series[i])
       return series
  
   def _cleaner(self,sent):
       cur = time.time()
       sent = sent.split()
       sent = [i.lower() for i in sent]
       sent = [i for i in sent if i not in self._sw]
       for i in sent:
           if 'http' in i or 'html' in i or 'www' in i or '.com' in i:
               sent.remove(i)
       sent = " ".join(sent)
       sent = list(sent)
       sent = [i for i in sent if not i.isdigit()]
       sent = "".join(sent)
       sent = sent.split()
       sent = " ".join(sent)
       sent = list(sent)
       sent = "".join([i for i in sent if i not in string.punctuation])  
       sent = sent.split()
       sent = [i for i in sent if not len(i) <= 2]
       end = time.time()
       print("Lower,Number removal,punc removal ","%.5f" % end-cur)
       cur = time.time()
       for i in sent:
           ind = sent.index(i)
           if i in self._double:
               continue
           else:
               i = self._remove_consec_duplicates(i)
               sent[ind] = i              
       for i in sent:
           if i not in self._eng_words:
               sent.remove(i)      
       end = time.time()
       print("http,double,conce ", "%.5f" % end-cur)              
       if self._stem:
           sent = [self._ps.stem(i) for i in sent]
       sent = " ".join(sent)
   
       return sent

   

In [411]:
cleaner = TextCleaning(False)

In [346]:
cleaner.clean(["List of text data"])